---
title: "Webscraping Indeed Job Portal"
description: "webscraping with python "
author: "Aakash Basnet"
date: "2024/02/03"
categories:
  - webscraping
  - code
  - ETL
  - python
format:
  html:
    code-fold: true
jupyter: python3
---

##  Building URL
After navigating the developer toolbar for Indeed job listing, I found the pattern in the url query for each job title search and location. We can use this info to build the url. The link printed from the code below will take you to the Indeed page having listing for python developer in Dalla, TX

In [ ]:
def url_builder(job_title, location, page_number=10 ):
    job_title = "+".join(job_title.split(" "))
    location = "+".join(location.split(" "))
    base_url = "https://www.indeed.com/jobs"
    query_str = f"?q={job_title}&l={location}"
    url = f"{base_url}{query_str}"
     
    return url

print(url_builder(job_title="python developer", location="Dallas, TX"))

## Scraping the indeep page with selenium
The script below scrapes the data for the given job title and location. It uses selenium web driver to automate the data scraping. The web driver clicks 'next' button on pagination until the end of the page.

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import time

def get_data(job_title, location):

    url = url_builder(job_title=job_title, location=location)
    driver = webdriver.Chrome()
    driver.get(url)

    jobs = []
    has_next = True
    count = 1
    while has_next:
        
        time.sleep(2)
        cards = driver.find_elements(By.CLASS_NAME,'cardOutline')
        for card in cards:
            job_title = card.find_element(By.CLASS_NAME,'jobTitle')
            job_title_text = job_title.text
            job_id = job_title.find_element(By.TAG_NAME, 'a').get_attribute('data-jk')
            location = card.find_element(By.CLASS_NAME,'company_location').text
            job_description = card.find_element(By.CLASS_NAME,'underShelfFooter').text
            
            try:
                pay,*_metadata = card.find_element(By.CLASS_NAME,'heading6').text.split('\n')
            except Exception as e:
                pay = 'NA'
                _metadata = []
        
            
            jobs.append({
                'job_id':job_id,
                'job_url': f"https://www.indeed.com/viewjob?jk={job_id}",
                'job_title':job_title_text,
                'location': location,
                'description': job_description,
                'pay rate': pay,
                'metadata': _metadata
            })

        try:
            driver.find_element(By.CSS_SELECTOR,"[data-testid='pagination-page-next']").click()
            count += 1
        except Exception as e:
            print(f"Ending at page {count}")
            has_next = False

    driver.close()
    return jobs



In [6]:
job_data = get_data(job_title='python developer', location='Fort Worth,TX')



Ending at page 35


In [7]:
import pandas as pd
df = pd.DataFrame(job_data)
df.head(40)

,job_id,job_url,job_title,location,description,pay rate,metadata
0,cd937da8c0f30efd,https://www.indeed.com/viewjob?jk=cd937da8c0f3...,Software Developer - AI Trainer (Contract),"DataAnnotation\n4.6\nRemote in Dallas, TX",DataAnnotation is committed to creating qualit...,$40 an hour,"[Contract, 1 to 40 hours per week, Choose your..."
1,dc3e79c2af261bbe,https://www.indeed.com/viewjob?jk=dc3e79c2af26...,Back End Developer,"IWP Services, LLC\nHybrid remote in Fort Worth...",Integration of user-facing elements developed ...,"$100,000 - $110,000 a year","[Full-time, +1, Monday to Friday, Employee sto..."
2,eabae05997a929fd,https://www.indeed.com/viewjob?jk=eabae05997a9...,Python Developer,"Robert Half\n3.9\nPlano, TX 75024",Periodically exercise code-review and band tog...,$63.34 - $73.34 an hour,[Temp-to-hire]
3,1ebb8e02e8eb7b07,https://www.indeed.com/viewjob?jk=1ebb8e02e8eb...,DevOps Engineer (309801),Internal Data Resources\n3.7\nRemote in Coppel...,IDR is seeking a DevOps Engineer to join one o...,$55 - $63 an hour,"[Full-time, 40 hours per week, 8 hour shift]"
4,8f182ee73fc19368,https://www.indeed.com/viewjob?jk=8f182ee73fc1...,Front End Web Developer,"PCI Enterprises\n4.3\nDallas, TX",We have a small team of developers meaning you...,"$65,000 - $75,000 a year","[Full-time, Monday to Friday, +1, Bonus opport..."
5,8d0dfc501ff3fb89,https://www.indeed.com/viewjob?jk=8d0dfc501ff3...,Python Developer,"Emonics LLC\nFort Worth, TX 76107 \n(Arlington...",8.Coordinating with front-end developers..\nTo...,"$70,000 - $130,000 a year","[Full-time, +1]"
6,693494f2bd167ac3,https://www.indeed.com/viewjob?jk=693494f2bd16...,Full Stack Engineer,"PCI Enterprises\n4.3\nDallas, TX",We have a small team of developers meaning you...,"$65,000 - $80,000 a year","[Full-time, Monday to Friday, +1, Bonus opport..."
7,8e5cb88d329ff384,https://www.indeed.com/viewjob?jk=8e5cb88d329f...,Software Developer,Boston Enterprises Investment Group LLC\nDeSot...,We are seeking a skilled Software Engineer Dev...,"$68,000 - $77,000 a year","[Full-time, +1, 8 hour shift]"
8,a2a8cbd917cbf446,https://www.indeed.com/viewjob?jk=a2a8cbd917cb...,Python Developer,"Qatalys Software Technologies\nIrving, TX",Analyzes business and technical requirements t...,,[]
9,91cdaee932850a17,https://www.indeed.com/viewjob?jk=91cdaee93285...,Software Engineer - Mid-Career (HYBRID TELEWORK),"Lockheed Martin Corporation\nFort Worth, TX","Date posted: Jan. 29, 2024.\nDescription:We ar...",Pay information not provided,"[Full-time, 4x10]"


In [8]:
df.shape

(518, 7)